In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import unquote
import re
import time
import requests
import urllib.request
import lxml

In [ ]:
driverPath = 'D:/book_social_network/chromedriver.exe'
driver = webdriver.Chrome(driverPath)

In [ ]:
titles = []
authors = []
descriptions = []
languages = []
pages = []
genres = []
publishers = []
isbn10 = []
isbn13 = []
dimensions = []
shipping_weight = []
published_date = []
book_images = []

In [ ]:
book_fields = [titles, 
               authors, 
               descriptions, 
               languages, 
               pages, 
               genres, 
               publishers, 
               isbn10, 
               isbn13, 
               dimensions, 
               shipping_weight, 
               published_date, 
               book_images
              ]

In [ ]:
# How many pages to loop through in a certain url
for i in range(2, 3):
    print("Current page is: " + str(i))
    driver.get("https://www.amazon.com/s?i=stripbooks&rh=n:283155,n:1000,n:5,n:3952&page=" + str(i))
    time.sleep(4)
    content = driver.page_source
    soup = BeautifulSoup(content)
    time.sleep(3)
    for div in soup.find_all('div', attrs = {'data-index':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}):
        links = div.find('a', attrs = {'class':'a-link-normal a-text-normal'})
        href_value = unquote(links.attrs.get('href'))
        book_url = 'https://www.amazon.com' + href_value
        driver.get(book_url)
        time.sleep(4)
        # Author identify code section
        author_href = div.find('a', attrs={'class':'a-size-base a-link-normal'})
        if (author_href):
            author_href_value = unquote(author_href.attrs.get('href'))
            author_regx = re.search('/e/B\w+\?', author_href_value)
            author_code = author_href_value[author_regx.start()+3:author_regx.end()-1]
        # Book publication date section
        published_div = div.find('div', attrs={'class':'sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-32 sg-col-12-of-20 sg-col-12-of-36 sg-col sg-col-12-of-24 sg-col-12-of-28'})
        book_publication = published_div.find('span', attrs={'class':'a-size-base a-color-secondary a-text-normal'}).text
        # Assign new data source
        book_content = driver.page_source
        book_soup = BeautifulSoup(book_content)
        time.sleep(3)
        # Center div book data to get
        book_middle_data = book_soup.find('div', attrs={'id':'centerCol'})
        # Title feature div
        title_feature_div = book_soup.find('div', attrs = {'id':'title_feature_div'})
        if (book_middle_data):
            # Book title section
            book_title = book_middle_data.find('span', attrs={'id':'productTitle'}).text
            # Book author section
            if (author_href == None):
                byline_info = book_middle_data.find('div', attrs = {'id':'bylineInfo'})
                book_author = byline_info.find('a', attrs = {'class':'a-link-normal'}).text
            else:
                book_author = book_middle_data.find('a', attrs = {'data-asin':author_code}).text
        else:
            # Book title section
            book_title = title_feature_div.find('span', attrs = {'id':'productTitle'}).text
            # Book author section
            if (author_href):
                book_author = title_feature_div.find('a', attrs = {'data-asin':author_code}).text
                # Data assignment section at the bottom
            else:
                byline_info = title_feature_div.find('div', attrs = {'id':'bylineInfo'})
                book_author = byline_info.find('a', attrs={'class':'a-link-normal'}).text
        # Book description section
        if (book_middle_data):
            book_desc = book_middle_data.find('noscript')
            if (book_desc == None):
                book_desc = "Null"
            else:
                book_desc = book_desc.get_text().strip()
        else:
            book_desc_feature = book_soup.find('div', attrs = {'id':'bookDescription_feature_div'})
            book_desc = book_desc_feature.find('noscript').get_text().strip()
        # Book genre section
            # Go to the data insert section
        # Book details section
        book_detail_table = book_soup.find('table', attrs={'id':'productDetailsTable'})
        ul_tag = book_detail_table.find('ul')
        item_rows = ul_tag.find_all('li')
        for item_row in item_rows:
            # Book page section
            if(re.search('^Paperback: ', item_row.text)):
                book_page_start = re.search('^Paperback: ', item_row.text)
                book_page_end = re.search(' pages', item_row.text)
                book_page = item_row.text[book_page_start.end():book_page_end.start()]
            # Book language section
            if (re.search('^Language: ', item_row.text)):
                book_language_regx = re.search('^Language: ', item_row.text)
                book_lang = item_row.text[book_language_regx.end():]
            # Book publisher section
            if (re.search('^Publisher: ', item_row.text)):
                book_publisher_regx = re.search('^Publisher: ', item_row.text)
                # Check whether the string match the (Month Date, Year)
                if (re.search(" \(\w+ \d+, \d+\)", item_row.text)):
                    published_date_regx = re.search(" \(\w+ \d+, \d+\)", item_row.text)
                    book_publisher = item_row.text[book_publisher_regx.end():published_date_regx.start()]
                # Check wether the string contains the symbol ; pattern
                elif (re.search(r';', item_row.text)):
                    string_spliter = re.search(r';', item_row.text)
                    book_publisher = item_row.text[book_publisher_regx.end():string_spliter.start()]
                elif (re.search(" \(\d+\)", item_row.text)):
                    year_publish = re.search(" \(\d+\)", item_row.text)
                    book_publisher = item_row.text[book_publisher_regx.end():year_publish.start()]
                else:
                    month_year = re.search(" \(\w+ \d+\)", item_row.text)
                    book_publisher = item_row.text[book_publisher_regx.end():month_year.start()]
            # Book isbn-10 section
            if (re.search("^ISBN-10: ", item_row.text)):
                book_isbn10_regx = re.search("^ISBN-10: ", item_row.text)
                book_isbn10 = item_row.text[book_isbn10_regx.end():]
                # For debug purpose
                print(book_isbn10)
            # Book isbn-13 section
            if (re.search("^ISBN-13: ", item_row.text)):
                book_isbn13_regx = re.search("^ISBN-13: ", item_row.text)
                book_isbn13 = item_row.text[book_isbn13_regx.end():]
            # Book dimensions section
            if (re.search("Product Dimensions: ", item_row.text)):
                book_dimension_start = re.search("Product Dimensions: ", item_row.text)
                book_dimension_end = re.search(' inches', item_row.text)
                book_dim = item_row.text[book_dimension_start.end():book_dimension_end.start()].strip()
            # Book weight
            if (re.search("^Shipping Weight: ", item_row.text)):
                book_weight_start = re.search("^Shipping Weight: ", item_row.text)
                book_weight_end = re.search(" \(View shipping rates and policies\)", item_row.text)
                if (book_weight_end):
                    book_weight = item_row.text[book_weight_start.end():book_weight_end.start()]
                else:
                    book_weight = item_row.text[book_weight_start.start():book_weight_start.end()]
        # Book image section
        if (book_middle_data):
            book_image_div = book_soup.find('div', attrs={'id':'booksImageBlock_feature_div'})
            image_scripts = book_image_div.find_all('script', attrs={'type':'text/javascript'})
            for image_script in image_scripts:
                if (re.search('\'imageGalleryData\' : \[\{"mainUrl":"https://images-na.ssl-images-amazon.com/images/I/[\Wa-zA-Z0-9]+.jpg', str(image_script))):
                    img_url_end = re.search('\'imageGalleryData\' : \[\{"mainUrl":"https://images-na.ssl-images-amazon.com/images/I/[\Wa-zA-Z0-9]+.jpg', str(image_script))
                    img_url_start = re.search('\'imageGalleryData\' : \[\{"mainUrl":"', str(image_script))
                    book_img_url = str(image_script)[img_url_start.end():img_url_end.end()]
        else:
            book_image_div = book_soup.find('div', attrs = {'id':'mainImageContainer'})
            book_image_tag = book_image_div.find('img', attrs = {'id':'imgBlkFront'}).attrs.get("src")
            book_img_url_regx = re.search(r"https://images-na.ssl-images-amazon.com/images/I/([a-zA-Z0-9+\-~$]*._)|([a-zA-Z0-9]*._)", book_image_tag)
            if (book_img_url_regx):
                book_img_url = book_image_tag[book_img_url_regx.start():book_img_url_regx.end()-2] + ".jpg"
        # Insert data into data list section
        titles.append(book_title)
        authors.append(book_author)
        descriptions.append(book_desc)
        languages.append(book_lang)
        pages.append(book_page)
        genres.append("computer_technology")
        publishers.append(book_publisher)
        isbn10.append(book_isbn10)
        isbn13.append(book_isbn13)
        dimensions.append(book_dim)
        shipping_weight.append(book_weight)
        published_date.append(book_publication)
        book_images.append(book_img_url)
    print("...Done!")

In [ ]:
csv_file_location = 'D:/amazon_book_dataset.csv'
df = pd.DataFrame({
    'Book titles': titles, 
    'Book authors': authors, 
    'Book descriptions': descriptions, 
    'Languages': languages, 
    'Pages': pages, 
    'Genres': genres, 
    'Publishers': publishers, 
    'ISBN_10': isbn10, 
    'ISBN_13': isbn13, 
    'Dimensions': dimensions, 
    'Shipping_weight': shipping_weight, 
    'Published_date': published_date, 
    'Book_image': book_images,
})
df.drop_duplicates()
df.to_csv(csv_file_location, index=False, encoding='utf-8', mode='a', header='False')